In [1]:
# randomNG = 'Philox'
# randomNG = 'PCG64DXSM'
# randomNG = 'MT19937'
# randomNG = 'SFC64'
import importlib

pass_abc ='xprobRRapril_MCD'

In [2]:
import numpy as np
import time
import datetime
import numba
import concurrent.futures
from math import lgamma, floor
import numpy.random
# bitNG = getattr(numpy.random, randomNG)
from numpy.random import Generator, SeedSequence
from ill_confirm_bias_FNs_03_t0FAST_03cnt_vdisc_explore_prob import confirm_bias_full
np.set_printoptions(suppress=True)



/home/nothin/python/confirm_bias_illusory_tuth_002/ill_confirm_bias_FNs_03_t0FAST_03cnt_vdisc_explore_prob.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit
/home/nothin/python/confirm_bias_illusory_tuth_002/ill_confirm_bias_FNs_03_t0FAST_03cnt_vdisc_explore_prob.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit
/home/nothin/python/co

In [3]:
# mp.set_start_method("fork")
start = time.perf_counter()

runs = 2000
numarms = 2
maxalphabeta = 4
# numagents = 50
# minagents = 4
# maxagents = 12
pulls = 1000
payprobs = np.asarray([.499, .5])
tollerance = 0
nsteps = 10
timesteps = 4*10**5

explprob = 0.001 #explore every expltstep many timesteps

mem_depth = [5, 10, 100, 1000]
mem_snap = [10, 100, 1000, 10000] #these values must be multiples of nsteps to get recorded


networktype = 'wheel'
erprobA = 0.5
erprobB = 2-2*(1-erprobA)**.5
print(f'erprobB = {erprobB}')

mem = 100 # must be greater than max numagents. FULL STOP? if memshare = 0
memshare = 30
memstore = 5
variablediscount = 1

#number of threads
thrds = 20
# numba.set_num_threads(thrds)

for randomNG in ['PCG64DXSM']:
    bitNG = getattr(numpy.random, randomNG)
    for numagents in [25]:
        sg = SeedSequence()
        rgs = [Generator(bitNG(s)) for s in sg.spawn(runs)]
        final_mp = np.zeros((runs, 2))
        depth_mp = np.zeros((runs, len(mem_snap)+1, len(mem_depth)))
        print('*********************************************')
        print('*********************************************')

        inputs = []

        with concurrent.futures.ProcessPoolExecutor(max_workers=thrds) as executor:
            future_to_confirm_bias_full = {executor.submit(confirm_bias_full, explprob, variablediscount, mem_snap, mem_depth, erprobB, mem, memshare, memstore, networktype, maxalphabeta, pulls, numagents, numarms, tollerance, payprobs, nsteps, timesteps, rgs, r): inputs for r in range(runs)}
            for future in concurrent.futures.as_completed(future_to_confirm_bias_full):
                inputs = future_to_confirm_bias_full[future]
                try:
                    data = future.result()
                except Exception as exc:
                    print(f'generated an exception: y? and {exc}')
                else:
                    final_mp[data[3]] = data[0:2]
                    depth_mp[data[3]] = data[2]


        final_mp = np.asarray(final_mp)
        suc_mp = np.asarray(final_mp[:,0])
        con_mp = np.asarray(final_mp[:,1])


        np.save(f'{pass_abc}expl{explprob}{randomNG}vdisc{variablediscount}t0fastNGconsensus_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', con_mp)
        np.save(f'{pass_abc}expl{explprob}{randomNG}vdisc{variablediscount}t0fastNGsuccess_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', suc_mp)
        np.save(f'{pass_abc}expl{explprob}{randomNG}vdisc{variablediscount}t0fastNGdepths_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', depth_mp)

        sucrate = np.sum(suc_mp)/runs
        conrate = np.sum(con_mp)/runs

        depth_mean = np.mean(depth_mp, axis=0)
        depth_std = np.std(depth_mp, axis=0)

        print('time:')
        print(datetime.datetime.now())

        print(f'M memory = {mem} S memshare = {memshare} D memstore = {memstore} discount factor = {variablediscount}')
        print(f'explore probability = {explprob}')
        print(f'for # agents = {numagents} success rate = {sucrate} consensus rate = {conrate}')
        print(f'using {randomNG} generator')
        print('network average memory depths:')
        print(depth_mean)
        print('depths standard deviation')
        print(depth_std)
        print(' ')
        print(' ')

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')


erprobB = 0.5857864376269049
*********************************************
*********************************************
time:
2024-04-13 22:57:42.378228
M memory = 100 S memshare = 30 D memstore = 5 discount factor = 1
explore probability = 0.001
for # agents = 25 success rate = 0.7635 consensus rate = 0.991
using PCG64DXSM generator
network average memory depths:
[[2485.92477949 2482.05144671 2413.43114159 1821.13949827]
 [   4.567         0.            0.            0.        ]
 [ 154.024       149.036         0.            0.        ]
 [2452.669      2448.8545     2379.204         0.        ]
 [2488.6765     2485.0325     2417.579      1827.2915    ]]
depths standard deviation
[[ 0.23940867  0.24493896  0.49397087  3.79052055]
 [ 8.35425107  0.          0.          0.        ]
 [79.17417144 78.6824358   0.          0.        ]
 [14.34961459 15.5982797  29.21496849  0.        ]
 [ 9.63129523 10.83556384 26.30484288 84.33403541]]
 
 
Finished in 282.45 minutes
